In [15]:
# !pip install nba_api

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import numpy as np
import pandas as pd
import tqdm
import glob
import os
import pickle

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import commonplayoffseries, playoffpicture, boxscoresummaryv2, leaguegamefinder,\
playbyplayv2, commonplayerinfo, commonallplayers
from nba_api.stats.library.parameters import Season, SeasonType, LeagueIDNullable, PerModeSimpleNullable,\
SeasonTypePlayoffs, SeasonSegmentNullable
from nba_api.stats.library.eventmsgtype import EventMsgType

from nba_api.stats.endpoints import BoxScoreMatchups, LeagueSeasonMatchups, PlayerDashPtShotDefend, PlayerDashboardByShootingSplits

pd.options.display.max_columns = 30
pd.options.display.max_rows = 100

/home/noone/.virtualenvs/env37/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
tt = PlayerDashboardByShootingSplits(player_id=1627783)#.get_dataframes()

In [4]:
rr = tt.get_data_frames()
len(rr)

8

In [9]:
rr[3]

,GROUP_SET,GROUP_VALUE,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,EFG_PCT,BLKA,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,...,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,EFG_PCT_RANK,BLKA_RANK,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK,CFID,CFPARAMS
0,Shot Area,Restricted Area,110,156,0.705,0,0,0.000,0.705,13,0.527,0.473,0.000,0.000,0.527,...,2,1,4,4,4,1,7,1,2,4,2,1,2,49,Restricted Area
1,Shot Area,In The Paint (Non-RA),77,177,0.435,0,0,0.000,0.435,11,0.260,0.740,0.000,0.000,0.260,...,1,2,4,4,4,2,6,2,1,4,2,3,1,49,In The Paint (Non-RA)
2,Shot Area,Mid-Range,44,120,0.367,0,0,0.000,0.367,3,0.182,0.818,0.000,0.000,0.182,...,3,3,4,4,4,3,5,3,3,4,2,4,3,49,Mid-Range
3,Shot Area,Left Corner 3,5,8,0.625,5,8,0.625,0.938,0,0.000,0.000,1.000,0.000,1.000,...,6,5,2,3,2,5,1,4,4,2,2,5,5,49,Left Corner 3
4,Shot Area,Right Corner 3,3,12,0.250,3,12,0.250,0.375,0,0.000,0.000,1.000,0.000,1.000,...,5,6,3,2,3,6,1,4,4,3,2,6,5,49,Right Corner 3
5,Shot Area,Above the Break 3,26,82,0.317,26,82,0.317,0.476,0,0.000,0.000,0.962,0.038,0.962,...,4,4,1,1,1,4,1,4,4,1,1,2,4,49,Above the Break 3
6,Shot Area,Backcourt,0,0,0.000,0,0,0.000,0.000,0,0.000,0.000,0.000,0.000,0.000,...,7,7,4,4,4,7,1,4,4,4,2,7,5,49,Backcourt


In [10]:
df = (
    pd.read_pickle('../active_players_seasons_stats_19012022.pickle.zip')
    .query("(SEASON_ID == '2021-22') and (FGA > 200)")
)
players = df.PLAYER_ID.unique()
len(players)

191

In [12]:
out_dir = '../shooting_splits_2021'
!mkdir -p $out_dir
!ls $out_dir | wc -l

0


In [14]:
for i, pl in enumerate(players):
    fname = os.path.join(out_dir, f'{pl}.pkl')
    if os.path.exists(fname):
        continue
    fr = PlayerDashboardByShootingSplits(player_id=pl,
                                         season='2021-22',
                                         season_type_playoffs='Regular Season',)
    with open(fname, 'wb') as f:
        pickle.dump(fr.get_data_frames(), f)
    if i % 20 == 0:
        print(i, end='  ')

20  40  60  80  100  120  140  160  180  